### Get the list of all resturant in Bangladesh using Google Place API
* []: # Language: python
* []: # Path: All_Bangladesh_Resturant_Google Place API.ipynb

### required libraries
* pandas
* requests
* time


In [ ]:
# imports
import pandas as pd
import requests
import time
# API key
API_KEY = 'YOUR-API-KEY'

### Get the places of all over Bangladesh from a public source and save it in a csv file for future use.
-----------------------------------------------------------------------------------------------------------------------
### Thanks To open source github repository for the places name and address.

In [3]:
# get places from places.csv
places = pd.read_csv('places.csv')
places.sample(5)

# @credit: https://github.com/ittahad/BDLocations


,division,district,subordinate,branch,post_code
1317,Rajshahi,Sirajganj,Sirajganj Sadar,Rashidabad,6702
348,Dhaka,Jamalpur,Dewangonj,Dewangonj,2030
623,Chattagram,Chittagong,Rouzan,Fatepur,4345
572,Chattagram,Chittagong,Chittagong Sadar,Chitt. Sailers Colon,4218
507,Chattagram,Noakhali,Noakhali Sadar,Noakhali Sadar,3800


In [4]:
# Table headings
keys = ['name','latitude','longitude','rating','number of reviews','affluence']


In [5]:
# function to extract name from the place result send None if not found
def getname(res):
    try:
        return res['name']
    except:
        return None

In [6]:
# function to extract latitude and longtitude from the place result send None if not found
def getlatlon(dic):
    try:
        lat = dic['geometry']['location']['lat']
        lon = dic['geometry']['location']['lng']
        return lat, lon
    except:
        return None, None

In [7]:
# function to extract rating from the place result send None if not found
def getrating(dic):
    try:
        rating = dic['rating']
        return rating
    except:
        return None

In [8]:
# function to extract user_rating_count from the place result send None if not found
def getuser_ratings_total(dic):
    try:
        user_ratings_total = dic['user_ratings_total']
        return user_ratings_total
    except:
        return None

In [9]:
# function to extract Price_level/affluence from the place result send None if not found
def getprice_level(dic):
    try:
        price_level = dic['price_level']
        indollarsign = "$"*int(price_level)
        return indollarsign
    except:
        return None

In [10]:
# function to get the data from the response result
def get_data(results):
    # add the data to the dataframe
    df = pd.DataFrame(columns=keys)
    for i in range(len(results)):
        dic = results[i]
        name = getname(dic)
        lat, lon = getlatlon(dic)
        rating = getrating(dic)
        user_ratings_total = getuser_ratings_total(dic)
        price_level = getprice_level(dic)
        # add the data to the dataframe
        df.loc[i] = [name, lat, lon, rating, user_ratings_total, price_level]
    return df

### Google place api return maximum 20 search result. So, we need to use pagination. Still only 60 total results can be returned.

In [11]:
def fetchdata(location):
    # function to fetch data from google place api
    url = f'''
    https://maps.googleapis.com/maps/api/place/textsearch/json?query=restaurants in {location}&type=restaurant&key={API_KEY}'''
    payload={}
    headers = {}
    response = requests.request("GET", url, headers=headers, data=payload)
    # string to dict
    res = response.json()
    return res

def fetchnextpage(nextpage_token):
    # function to fetch next page data from google place api using nextpage_token
    url = f'''
    https://maps.googleapis.com/maps/api/place/textsearch/json?pagetoken={nextpage_token}&key={API_KEY}'''
    payload={}
    headers = {}
    response = requests.request("GET", url, headers=headers, data=payload)
    # string to dict
    res = response.json()
    return res

In [12]:
# declare the dataframe
df = pd.DataFrame(columns=keys)

In [13]:
def crawl(location):
    global df
    # start the crawler
    res = fetchdata(location)
    # create a dataframe
    temp_df = get_data(res['results'])
    df = df.append(temp_df)
    time.sleep(5)
    for x in range(3):
        try:
            res = fetchnextpage(res['next_page_token'])
            temp_df = get_data(res['results'])
            df = df.append(temp_df)
            df.reset_index(drop=True, inplace=True)
            time.sleep(5)
        except Exception as e:
            # print(e)
            break

In [ ]:
for i in range(len(places)):
# for i in range(10):
    location = places.iloc[i]['branch']+', '+places.iloc[i]['subordinate']+', '+places.iloc[i]['district']+', '+places.iloc[i]['division']
    # print(location)
    crawl(location)
    # print(df.shape)

In [17]:
# drop the duplicates based on the name and lat and lon
df.drop_duplicates(subset=['name','lat','lng'], inplace=True, keep='first', ignore_index=True)


In [19]:
df.to_csv('restaurant_data.csv', index=False)